In [18]:
# %pip install openai
# %pip install --upgrade openai

In [2]:
import pandas as pd

dataset_dir = 'datasets'

# read the dataset
df = pd.read_csv(f'{dataset_dir}/podcast.csv')
df.head()

,text,text_length,sentiment,word_count,unique_word_count,pos_tags,entities,clean_text
0,"As part of MIT course 6S099, Artificial Genera...",74068,0.171118,13424,2726,"[('As', 'IN'), ('part', 'NN'), ('of', 'IN'), (...","[('MIT', 'ORG'), ('Artificial General Intellig...",part mit course artificial general intelligenc...
1,As part of MIT course 6S099 on artificial gene...,57788,0.116574,10217,2474,"[('As', 'IN'), ('part', 'NN'), ('of', 'IN'), (...","[('MIT', 'ORG'), ('Christoph Koch', 'PERSON'),...",part mit course artificial general intelligenc...
2,"You've studied the human mind, cognition, lang...",33940,0.106534,5989,1762,"[('You', 'PRP'), (""'ve"", 'VBP'), ('studied', '...","[('Plato', 'ORG'), ('B.', 'NORP'), ('Nietzsche...",studied human mind cognition language vision e...
3,What difference between biological neural netw...,34228,0.106092,5993,1528,"[('What', 'WP'), ('difference', 'NN'), ('betwe...","[('First', 'ORDINAL'), ('One', 'CARDINAL'), ('...",difference biological neural networks artifici...
4,The following is a conversation with Vladimir ...,36171,0.125654,6374,1481,"[('The', 'DT'), ('following', 'NN'), ('is', 'V...","[('Vladimir Vapnik', 'PERSON'), ('VC', 'PERSON...",following conversation vladimir vapnik co inve...


In [20]:
# we only want the text column
df = df[['text']]
df = df.dropna()

In [ ]:
from openai import OpenAI

api_key = ""

client = OpenAI(
    api_key=api_key,
)

In [39]:
prompt = "Write a short summary, limited to 40 words, for the following long form text: "

def generate_summary(text, index):
    # gpt3 has a maximum context length is 16385 tokens.
    # I guesstimated the characters that allow for 16385 tokens
    max_text_length = 65000
    if len(text) > max_text_length:
        text = text[:max_text_length]
    try:
        prompt_to_send = prompt + "\"" + text + "\""
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt_to_send,
                }
            ],
            model="gpt-3.5-turbo",
        )
        response_summary = chat_completion.choices[0].message.content
        # remove "Summary:" from the response
        response_summary = response_summary.replace("Summary:", "")
        return response_summary
    except Exception as e:
        print(f"Error generating summary for text: {e}, Index {index}")
        print(f"Text Length: {len(text)}")
        return None

# generate row 1 summary
print(generate_summary(df['text'][126], 126))

df['summary'] = df['text'].apply(lambda x: generate_summary(x, df['text'].index[df['text'] == x][0]))

 have to sit down and put in the work to cultivate those ideas and turn them into something meaningful. It's a balance of receiving inspiration and putting in the effort to bring that inspiration to life. And that's what creativity is all about. It's a magical process of ideas finding you and you nurturing them to reality. And that's the beauty of it all.


In [40]:
df.head()

,text,summary
0,"As part of MIT course 6S099, Artificial Genera...",AGI as described by MIT students revolve aroun...
1,As part of MIT course 6S099 on artificial gene...,Neurobiologist Christoph Koch discusses the st...
2,"You've studied the human mind, cognition, lang...","Stephen Pinker discusses the meaning of life, ..."
3,What difference between biological neural netw...,The text explores the mysteries of biological ...
4,The following is a conversation with Vladimir ...,"Vladimir Vapnik, co-inventor of support vector..."


In [ ]:
# save the dataset
df.to_csv(f'{dataset_dir}/podcast_with_summary.csv', index=False)

In [4]:
# also save a train and test dataset
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)

train.to_csv(f'{dataset_dir}/podcast_with_summary_train.csv', index=False)
test.to_csv(f'{dataset_dir}/podcast_with_summary_test.csv', index=False)